<a href="https://colab.research.google.com/github/felixsimard/comp551-p4/blob/main/Felix_Extreme_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## COMP 551: Applied Machine Learning
### **P4 - Reproducibility in ML**

Charles Bourbeau (260868653) <br>
Mathis Renier () <br>
Felix Simard (260865674) <br>

Dec 10th, 2021



In [1]:
!pip install regex requests hydra-core omegaconf bitarray

In [2]:
# !git clone https://github.com/pytorch/fairseq
# %cd /content/fairseq
# !pip install --editable ./
# %cd /content
!pip install fairseq

In [68]:
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary
import torch.nn.functional as F

from fairseq.modules.quantization.pq import quantize_model_, SizeTracker


from operator import attrgetter, itemgetter
import re

In [4]:
# roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
# roberta.eval()

In [62]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [148]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, (3,3))
        self.conv2 = nn.Conv2d(6, 16, (3,3))
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [149]:
efficient_net_b3 = models.efficientnet_b3().to(device)
# summary(efficient_net_b3, (3, 256, 256))

vgg11 = models.vgg11().to(device)
# summary(vgg11, (3, 256, 256))

In [150]:
# from fairseq.modules.quant_noise import quant_noise
# quant_module = quant_noise(vgg11, p=0.1, block_size=8)
# quant_module

In [151]:
config = {
    'n_centroids': {
              'Conv2d': ('kernel_size', {'*': 256}),
              'Linear': ('in_features', {'*': 256})
          },
    'block_sizes': {
              'Conv2d': ('kernel_size', {'(3, 3)': 9, '(1, 1)': 4}), # '(3, 3)': 9
              'Linear': ('in_features', {'*': 4})
          },
    'layers_to_quantize': [".*?"]
}

model = net

In [152]:
# layers = [module for module in model.modules() if isinstance(module, (nn.Conv2d))]
# print(type(layers))
# print(len(layers))
# print(layers[0])
# print(layers[:3])

In [153]:
# all_layers = map(itemgetter(0), model.named_parameters())
# print(list(all_layers))
# filter_regexp = "(module\\.)?"
# r = re.compile(filter_regexp)
# list(filter(r.match, all_layers))

In [154]:
# for m in model.named_parameters():
#     print(m)

In [155]:
# get configuration parameters
n_centroids_config = config["n_centroids"]
block_sizes_config = config["block_sizes"]
layers_to_quantize = ".*?"

In [156]:
size_tracker = SizeTracker(model)

In [157]:
import logging

logger = logging.getLogger()

for step in range(len(layers_to_quantize)):
    quantized_layers = quantize_model_(
        model,
        size_tracker,
        layers_to_quantize,
        block_sizes_config,
        n_centroids_config,
        step=step
    )
    logger.info(layers_to_quantize[step])
    logger.info(f"Finetuning stage {step}, quantized layers: {quantized_layers}")
    logger.info(f"{size_tracker}")

    # Don't forget to re-create/update trainer/optimizer since model parameters have changed
    # optimizer = 

    # Finetune the centroids with your usual training loop for a few epochs
    # trainer.train_epoch()

2021-12-06 01:45:59 | INFO | root | Quantizing layer conv1 with block size 9 and 256 centroids
2021-12-06 01:45:59 | INFO | root | Could not resolve all empty clusters, 250 remaining
2021-12-06 01:45:59 | INFO | root | Quantizing layer conv2 with block size 9 and 256 centroids
2021-12-06 01:46:00 | INFO | root | Could not resolve all empty clusters, 160 remaining
2021-12-06 01:46:00 | INFO | root | Quantizing layer fc1 with block size 4 and 256 centroids
2021-12-06 01:46:01 | INFO | root | Iteration: 0,	objective: 1.741899,	resolved empty clusters: 2
2021-12-06 01:46:02 | INFO | root | Iteration: 1,	objective: 1.630717,	resolved empty clusters: 0
2021-12-06 01:46:02 | INFO | root | Iteration: 2,	objective: 1.593857,	resolved empty clusters: 0
2021-12-06 01:46:03 | INFO | root | Iteration: 3,	objective: 1.573702,	resolved empty clusters: 0
2021-12-06 01:46:03 | INFO | root | Iteration: 4,	objective: 1.560260,	resolved empty clusters: 0
2021-12-06 01:46:04 | INFO | root | Iteration: 5,	o

error: ignored